# Data Processing

The client provided the data in an excel sheet - `data/main.xlsx`. There are five sheets (seawater, copper, cadmium, lead, and mix) but we didn't work with mix because we wanted to focus on building a simple model first. 

Each row of each sheet is a water sample and the output of the readings when the volts were passed through them. 

My first goal was to get everything in a format I am more familiar with: csv files and pandas dataframes. 

## The Process

1. Extract the sheets from the excel file into dataframes
2. Drop unnecessary columns
3. Create unique, descriptive column names for each sample (including metal, concentration and sample number)
4. Transfrom the dataframes from wide-form to long-form
5. Create a voltage column
6. Reset index to be unique

It's all in the `data.py` file - rename it to data_processing or something. 

In [7]:
import numpy as np
import pandas as pd
from pathlib import Path

DATA_DIR = Path('data')

In [8]:
# Read in the sheets we want from the Excel file
sheet_names = ['Seawater - No Heavy Metals', 'Copper', 'Cadmium', 'Lead']
xcel = pd.read_excel(DATA_DIR / 'main.xlsx', sheet_name=sheet_names)

# Create dataframes for each class
seawater = xcel['Seawater - No Heavy Metals']
copper = xcel['Copper']
cadmium = xcel['Cadmium']
lead = xcel['Lead']

In [9]:
copper.columns

Index([         'Name',       'Analyte', 'Concentration',               1,
                 0.996,           0.992,           0.988,           0.984,
                  0.98,           0.976,
       ...
             '0.964.1',       '0.968.1',       '0.972.1',       '0.976.1',
              '0.98.1',       '0.984.1',       '0.988.1',       '0.992.1',
             '0.996.1',           '1.1'],
      dtype='object', length=1005)